In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [21]:
import numpy as np
import os.path as osp
import matplotlib.pylab as plt
from sklearn.manifold import TSNE

from general_tools.in_out.basics import unpickle_data
from general_tools.in_out import create_dir
from general_tools.notebook.tf import reset_tf_graph

from general_tools.clustering.plt import plot_2d_embedding_in_grid_greedy_way, \
                                         plot_2d_embedding_in_grid_forceful

from general_tools.plotting import stack_images_horizontally, stack_images_in_square_grid

from geo_tool import Point_Cloud

import tf_lab.point_clouds.in_out as pio

from tf_lab.data_sets.shape_net import  pc_loader as sn_pc_loader

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.in_out.basics import read_saved_epochs
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from sklearn.ensemble import IsolationForest
from sklearn.metrics.pairwise import pairwise_distances

from general_tools.clustering.dist import incremental_farthest_sampling, evaluate_solution
from general_tools.in_out.basics import files_in_subdirs

from harnessing_geometric_triplets import make_triplets, save_triplets, plot_triplets

In [3]:
def detect_outliers(features, outlier_fraction, random_seed):
    '''
    '''
    clf = IsolationForest(max_samples=100, random_state=random_seed, contamination=outlier_fraction)
    clf.fit(features)
    y_pred = clf.predict(features)
    outliers = np.where(y_pred == -1)[0]
    print 'Outliers detected = ', len(outliers)
    return outliers

In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
random_seed = 42
outlier_fraction = 0.3 # Percent of data instances that will be automatically disreguarded.
n_exemplars = 140      # Source instances that will be used to create contrasting triplets.
top_data_dir = top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [6]:
# load AE model
class_name = 'chair'
syn_id = snc_category_to_synth_id()[class_name]
ae_loss = 'emd'
ae_id = '12' # 128 bottleneck

ae_net_name = 'ae_farm_' + class_name + '_conv_arch_' + ae_id + '_2048pts_' + ae_loss
ae_net_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_net_name)
ae_conf = Conf.load(osp.join(ae_net_dir, 'configuration'))

if ae_conf.train_dir != ae_net_dir:
    ae_conf.train_dir = ae_net_dir
    ae_conf.save(osp.join(ae_conf.train_dir, 'configuration'))
    
reset_tf_graph()
ae = PointNetAutoEncoder(ae_net_name, ae_conf)

saved_epochs = read_saved_epochs(ae_conf.train_dir)
load_epoch = saved_epochs[-1]
ae.restore_model(ae_conf.train_dir, load_epoch, verbose=True)

Model restored in epoch 1000.


In [7]:
# Load Raw Point-Clouds of class
n_pc_samples = 2048
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = [f for f in files_in_subdirs(pclouds_path, '.ply')]
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )

6778 files containing complete point clouds were found.


In [8]:
pcloud_data = PointCloudDataSet(pclouds, labels=model_ids, init_shuffle=False)
_, latent_codes, temp_names = ae.embedding_at_tensor(pcloud_data, ae_conf, tensor_name='bottleneck')
assert(np.alltrue(temp_names==model_ids))

In [9]:
top_image_dir = '/orions4-zfs/projects/optas/DATA/Meshes/Shape_Net_Core/2015_Summer_OUT/Images/'
top_image_dir = osp.join(top_image_dir, syn_id)
image_view_tag = 'image_p020_t337_r005.png'
top_out_dir = '/orions4-zfs/projects/optas/DATA/OUT/3d_neighbs_as_context/pilot_triplets_2/'
top_out_dir = osp.join(top_out_dir, 'outlier_frac_' + str(int(outlier_fraction * 100)))
create_dir(top_out_dir)

'/orions4-zfs/projects/optas/DATA/OUT/3d_neighbs_as_context/pilot_triplets_2/outlier_frac_30'

In [10]:
all_pdists = pairwise_distances(latent_codes)
outliers = detect_outliers(latent_codes, outlier_fraction, random_seed)
fps_sol = incremental_farthest_sampling(all_pdists, n_exemplars, outliers, random_seed)
print 'Average Min Distance of FPS exemplars:', evaluate_solution(latent_codes[fps_sol])

Outliers detected =  2034
Average Min Distance of FPS exemplars: 319.710628986


In [45]:
# pdists_ = np.triu(all_pdists, 0) 
# pdists_ = pdists_[pdists_ > 0]
too_close_threr = np.percentile(pdists_, 0.01)


In [ ]:
for i in range(len(pclouds)):
    if all_pdists[i, :]

In [44]:
# np.min(all_pdists[outliers[3], outliers[1:]])

# all_pdists[triplets[21][0], triplets[21][1]]

np.any(outliers == triplets[21][2])

False

In [11]:
plot_tsne = False
if plot_tsne:
    # Plot TSNE of examplars.
    model = TSNE(n_components=2, random_state=random_seed, init='pca')
    tsne_coords = model.fit_transform(latent_codes[fps_sol])

    # Make a list with the file-names of all to be used images.
    image_files = []
    for i in fps_sol:
        image_files.append(osp.join(top_image_dir, model_ids[i], image_view_tag))

    save_file = osp.join(top_out_dir, 'tsne_fps_%d_examplars.png' % (n_exemplars, ))
    small_dim = 400
    big_dim = int(np.ceil(np.sqrt(n_exemplars))) * small_dim
    plot_2d_embedding_in_grid_forceful(tsne_coords, image_files, big_dim, small_dim, save_file=save_file);

In [20]:
rule = 'closest_nn'
triplets = make_triplets(fps_sol, all_pdists, outliers, rule)
triplet_out_dir = create_dir(osp.join(top_out_dir, rule, 'data'))
save_triplets(triplets, top_image_dir, triplet_out_dir, model_ids)
triplet_out_dir = create_dir(osp.join(top_out_dir, rule, 'images'))
plot_triplets(triplets, top_image_dir, triplet_out_dir, model_ids)

[2725, 4016, 4260]

In [28]:
triplet_out_dir = create_dir(osp.join(top_out_dir, rule))
triplets = []
for rule in ['closest_nn', 'one_far', 'both_far']:
    triplets.append(make_triplets(fps_sol, all_pdists, outliers, rule))

triplet_out_dir = create_dir(osp.join(top_out_dir, 'all_rules', 'images'))
plot_triplets_in_multiple_contexts(triplets, top_image_dir, triplet_out_dir, model_ids)